# js_proxy

The `js_proxy` widget implementation is meant to be the widget to end all widgets (sort of).

To implement a Jupyter/Python/javascript widget you generally need a Python side implementation
for the widget and a javascript side for the widget which match up to eachother.  In general to
add more features to the widget you need to modify both sides in matching ways.

For high performance widgets that is the right way to go because it allows you to maintain
complex state on the javascript side and handle high performance callbacks and other operations
without requiring communication between the python side and the javascript side (which may
be very important especially if the two sides are running on different computers over a network
connection).

However for widgets where the performance considerations are not so critical the widget framework
can be cumbersome.  For example if I want to have jQueryUI dialog pop up I need to execute something
along the lines of the following javascript:

`$(DomElement).html("Hello from jQueryUI").dialog()`

Using the `js_proxy` mechanism you can do something like this that looks similar:

In [36]:
# First import needed modules and javascript support
from jp_gene_viz import js_proxy
# this loads the proxy widget javascript "view" implementation
js_proxy.load_javascript_support()
from IPython.display import display

In [37]:
# Then create a "proxy widget" for the jQueryUI dialog.
d = js_proxy.ProxyWidget()

# Construct a command to make the widget into a jQueryUI dialog.
command = d.element().html('<b id="my_dialog">Hello from jQueryUI</b>').dialog()

# Send the command to the widget view (javascript side).
d.send(command)

# Display the widget, which causes the command to also execute.
display(d)

If you execute the previous cell you should see a jQueryUI dialog pop up.
Click the close button to make the dialog invisible.

The following cell should make it appear again.

In [38]:
make_visible = d.element().dialog()
d.send(make_visible)

[1, [['method', ['element'], 'dialog']], 1]

The above illustrates that commands can be sent to the widget after it has been displayed.

[It is important to note that no commands are executed before a widget is displayed, so
until the widget is displayed you should only call `send` or `send_commands` (which sends
multiple commands in one go) once, because only the commands from the last call will get
executed.  After the widget is displayed multiple calls are allowed.]

The proxy mechanism works by capturing interactions on the Python side, encoding those
interactions as a JSON object, transferring the JSON object to the Javascript side, and then
interpreting the JSON object as a sequence of actions on the Javascript side.

Results from the Javascript actions are also passed back to the python side in the `results` 
trait of the Python side widget object:

In [39]:
d.results  # The results from the last command are not particularly meaningful.

[1, [{u'jquery': u'2.0.3', u'length': 1, u'selector': u''}]]

# Getting values from Javascript

The communication between the Python widget object and the Javascript view is asyncronous.
To do something with the results generated by a javascript interaction you may associate a
callback to process the results when they arrive.

In [40]:
# We want to put the html from the widget in this list
save_list = []

def save_command_result(result):
    "this is the callback we want to execute when the results arrive"
    #print (result)
    save_list.append(result)

# This "action" gets the html content of the widget.
get_html = d.element().html()

# Send the action to the javascript side for async execution.
d.send(get_html, results_callback=save_command_result)

# If we look at save_list now, it will probably be empty because the
# javascript side has probably not responded yet.
save_list

[]

In [41]:
# But later we should see the HTML saved in the list.
save_list

[[u'<b id="my_dialog">Hello from jQueryUI</b>']]

The result returned to the callback shown above 
is the HTML for the widget inside a list.
The result is in a list because in general the callback mechanism
permits results for command sequences in addition to results of 
individual commands.


# Command result translation depth level

Results of commands are translated to JSON compatible values using recursive traversal
of Javascript arrays and objects truncated to a depth which defaults to 1.  To traverse
deeper into the structures you may provide a `level` parameter for `send` or `send_commands`
which allows recursive depth up to 5.  To prevent recursion into complex values
set the `level` to `0`.

Below we get a fairly large data structure from the HTML Document Object Model (DOM) tree
by traversing the DOM element for the widget to 2 levels deep.

In [42]:
# get the DOM element associated with the widget from inside the JQuery container.
get_dom_element = d.element().get(0)
d.send(get_dom_element, results_callback=save_command_result, level=2)

[3, [['method', ['element'], 'get', 0]], 2]

In [43]:
# examine the JSON representation for the element sent to the callback (up to 2 levels deep)
save_list[-1]

[{u'ATTRIBUTE_NODE': 2,
  u'CDATA_SECTION_NODE': 4,
  u'COMMENT_NODE': 8,
  u'DOCUMENT_FRAGMENT_NODE': 11,
  u'DOCUMENT_NODE': 9,
  u'DOCUMENT_POSITION_CONTAINED_BY': 16,
  u'DOCUMENT_POSITION_CONTAINS': 8,
  u'DOCUMENT_POSITION_DISCONNECTED': 1,
  u'DOCUMENT_POSITION_FOLLOWING': 4,
  u'DOCUMENT_POSITION_IMPLEMENTATION_SPECIFIC': 32,
  u'DOCUMENT_POSITION_PRECEDING': 2,
  u'DOCUMENT_TYPE_NODE': 10,
  u'ELEMENT_NODE': 1,
  u'ENTITY_NODE': 6,
  u'ENTITY_REFERENCE_NODE': 5,
  u'NOTATION_NODE': 12,
  u'PROCESSING_INSTRUCTION_NODE': 7,
  u'TEXT_NODE': 3,
  u'accessKey': u'',
  u'addEventListener': {},
  u'align': u'',
  u'animate': {},
  u'appendChild': {},
  u'attributes': {u'length': 3},
  u'baseURI': u'http://localhost:8888/notebooks/repos/jp_gene_viz/examples/js_proxy%20example.ipynb#',
  u'blur': {},
  u'childElementCount': 1,
  u'childNodes': {u'length': 1},
  u'children': {u'length': 1},
  u'classList': {u'0': u'ui-dialog-content',
   u'1': u'ui-widget-content',
   u'length': 2},
  u

# Events

The proxy widget method 

`w.callback(self, callback_function, data, level=1)`

Sets up the infrastructure for a javascript callback which can communicate
a callback event notification from Javascript to the Python interpreter.

For example the following code creates an input element with
"calendar datepicker" jQueryUI functionality and also associates
a callback to be called when the input value changes.

In [44]:
# Create the widget.
dp = js_proxy.ProxyWidget()

# Command to populate the widget with an input element with id dp000.
make_input = dp.element().html('<input type="text" id="dp000"/>')._null()

# Command to make the input element into a datepicker and
# fix the style so the datepicker sits on top of the notebook page.
fix_style = (
    dp.window().
    jQuery("#dp000").  # get the jQuery input element by id == "dp".
    datepicker().   # make it a jQuery UI datepicker.
    css("position", "relative").
    css("z-index", "10000").  # put it on top
    attr("size", 55).  # make it big.
    _null()   # we don't care about the command result, discard it.
    )

# Define a python function and data structures to capture
# values sent to the callback when the datepicker input value changes.
identifiers_list = []
arguments_list = []

def dp_change_handler(identifier, arguments):
    "Print the results and also store them in lists."
    print (identifier, arguments['0']['target']['value'])
    identifiers_list.append(identifier)
    arguments_list.append(arguments)
    
# Command to create a "proxy callback" for the change event.
# The proxy will translate values to JSON up to 3 levels deep
# and also send the identifier data "dp has changed" to the handler.
proxy_callback = dp.callback(dp_change_handler, data="dp has changed", level=3)

# Command to associate the proxy callback with the datepicker change event
# using the standard $(x).change(callback) jQuery method.
on_change_command = dp.window().jQuery("#dp000").change(proxy_callback)

# Send the commands to the Javascript view.
dp.send_commands([make_input, fix_style, on_change_command])

# display the widget
display(dp)

(u'dp has changed', u'11/06/2015')


When you change the value in the datepicker input box
you should see print statements appear from the `dp_change_handler` callback.

# Other notes

The cell that created the datepicker above deserves additional explanation.

* `._null()` at the end of a chain of proxy commands indicates that the result of the command should be discarded.
The result for that command sent back to Python will be `None`.  This is useful to prevent communication of
large datastructures that are not needed.

* The `dp.window()` proxy command corresponds to the browser window namespace.
This allows the proxy widget access to global names in the browser environment such
as `dp.window().jQuery` which identifies the jQuery object.

* The `data` given to the proxy callback may be used to for cross referencing events
with other data structures (without having to define a new callback for every event).

* The `proxy_callback` is set to translate JSON values 3 levels deep because the
arguments sent to the callback from javascript holds the changed input value
`arguments['0']['target']['value']`, which is 3 levels deep.

* Callbacks functions registered using `w.callback(callback_function)` are stored
forever in an internal data structure `w.identifier_to_callback` unless the
the callback function is explicitly deleted with `w.forget_callback(callback_function)`.

* In the code we use `dp.window().JQuery("#dp000")` multiple times for simplicity.  It is also
possible to use a variable `dp000 = dp.window().JQuery("#dp000")` as a shorthand.

# Storing javascript values in the "element"

It is often important to store data structures created in Javascript for later
use.  Any javascript value which can be identified or created by the proxy mechanism
can be stored into the `element` object using the `_set` method.  For example the following interaction
creates a new `input` element and stores it in the `element` object namespace.

In [45]:
document = dp.window().document
new_input = document.createElement("input")
save_input = dp.element()._set("saved_input", new_input)
dp.send(save_input)

[2,
 [['set',
   ['element'],
   'saved_input',
   ['method', ['get', ['window'], 'document'], 'createElement', 'input']]],
 1]

In [46]:
# what is the type of the new input element?
dp.send(dp.element().saved_input.type)

[3, [['get', ['get', ['element'], 'saved_input'], 'type']], 1]

In [47]:
# apparently the default type for an input element is "text"
dp.results

[3, [u'text']]